In [1]:
import pandas as pd
import numpy as np
import os
import time
import cv2
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import random

In [2]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense
from keras.layers import Flatten, Activation, Concatenate
from keras.losses import categorical_crossentropy
from keras.optimizers import adam

Using TensorFlow backend.


In [3]:
FOLDER = '../../data_kaggle/kuzushiji/'
IMAGES = FOLDER + 'train_images/'
CHARS = FOLDER + 'chars/'
MODELS = FOLDER + 'models/'
print(os.listdir(FOLDER))

['unicode_translation.csv', 'train_images', 'sample_submission.csv', 'train.csv', 'kuzushiji-recognition.zip', 'output', 'test_images', 'chars', 'models']


In [4]:
img_size = (1000, 1000, 1)
inputs = Input(shape=img_size, name='ssd_input')

## Block 1
conv1_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_1')(inputs)
conv1_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_2')(conv1_1)
pool1 = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool1')(conv1_2)

conv2_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv2_1')(pool1)
conv2_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv2_2')(conv2_1)
pool2 = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool2')(conv2_2)

conv3_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv3_1')(pool2)
conv3_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv3_2')(conv3_1)
pool3 = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool3')(conv3_2)
flat3 = Flatten(name='flat3')(pool3)

dense1 = Dense(4787,activation='relu', name='ssd_dense1')(flat3)
dense2 = Dense(4787,activation='relu', name='ssd_dense2')(dense1)
dense3 = Dense(4787, activation='softmax', name='ssd_dense3')(dense2)
model =  Model(inputs, dense3)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ssd_input (InputLayer)       (None, 1000, 1000, 1)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 1000, 1000, 64)    640       
_________________________________________________________________
conv1_2 (Conv2D)             (None, 1000, 1000, 64)    36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 500, 500, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 500, 500, 64)      36928     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 500, 500, 64)      36928     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 250, 250, 64)      0         
__________

In [6]:
model.compile(optimizer=adam(),
              loss=categorical_crossentropy)

In [7]:
model.layers

In [8]:
model.layers[1].get_weights()

ResourceExhaustedError: OOM when allocating tensor with shape[1000000,4787] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: ssd_dense1/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=1216198, _device="/job:localhost/replica:0/task:0/device:GPU:0"](ssd_dense1/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'ssd_dense1/random_uniform/RandomUniform', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ryoji/Documents/git/kaggle/ml/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ryoji/Documents/git/kaggle/ml/lib/python3.6/site-packa

In [9]:
model.load_weights(MODELS + 'model.hdf5', by_name=True)

In [10]:
model.layers[1].get_weights()

[array([[[[-9.03471261e-02, -7.80396909e-02,  7.59378821e-02,
           -6.98826537e-02,  1.04765315e-02, -2.05819815e-01,
            3.47533189e-02,  1.29910959e-02,  3.67937125e-02,
            2.18261387e-02,  1.35330781e-01, -5.42032858e-03,
           -1.33986235e-01,  6.25160560e-02, -2.93885656e-02,
           -2.13474128e-02, -1.47271797e-01,  1.17946468e-01,
            5.37120178e-02, -1.41134575e-01, -3.94601040e-02,
           -2.83158541e-01, -7.59595037e-02, -1.08263046e-01,
           -1.58895403e-01,  5.81784546e-02, -3.61978151e-02,
           -4.09806259e-02, -2.06458107e-01, -3.80582288e-02,
           -1.03195235e-01,  7.70895407e-02, -2.86785781e-01,
            7.66610447e-03,  4.42925729e-02, -6.15289770e-02,
           -6.00584270e-03,  2.69304197e-02, -7.30878860e-02,
            6.51855543e-02, -7.88988546e-03,  1.40432589e-04,
           -6.48645312e-02,  7.78460503e-02, -3.34806666e-02,
           -3.79321501e-02,  3.49075086e-02, -9.89117771e-02,
        